## 完成図

プログラムが表す写真を修論から取ってくる

上から見た時の図を表すプログラムを書いている

opening_widthを徐々に大きくして狭い範囲を繰り返し露光することでだんだん広げる


In [28]:
# 単位はµm
import math


def compute_rectangles(device_width, margin_bottom, polarization_period, one_shot_length, opening_width):
    assert opening_width < polarization_period, "polarization_period must be greater than opening_width"
    polarizations_count = math.floor(one_shot_length / polarization_period)
    rectangles = []
    for n in range(polarizations_count):
        rectangle = {
            'top_left': [n * polarization_period, 0],
            'bottom_right': [n * polarization_period + opening_width, device_width - margin_bottom],
        }
        rectangles.append(rectangle)
    return rectangles


def do_compute_rectangles(polarization_period):
    device_width = 500
    # レジストが正しい形で堆積されているか確認するための余白
    margin_bottom = 10  # TODO: 実際の数値を修論参考にして考える

    opening_width = 0.5  # TODO: 修論で実際の数値を確認する
    one_shot_length = 0.5 * 1000

    rectangles = compute_rectangles(
        device_width=device_width,
        margin_bottom=margin_bottom,
        polarization_period=polarization_period,
        one_shot_length=one_shot_length,
        opening_width=opening_width,
    )

    return rectangles

## 確認用のCSVファイルを生成する

In [27]:
def compute_csv_row(rectangle):
    return f"{rectangle['top_left'][0]}, {rectangle['top_left'][1]}, {rectangle['bottom_right'][0]}, {rectangle['bottom_right'][1]}"


def generate_csv(rectangles):
    filename = input("Enter csv filename: ")
    rows = []
    for rectangle in rectangles:
        rows.append(compute_csv_row(rectangle=rectangle))
    with open(f"drawing_config/{filename}.csv", "w") as f:
        f.writelines(rows)


def do_generate_csv():
    polarization_period = 2.2
    rectangles = do_compute_rectangles(polarization_period=polarization_period)
    generate_csv(rectangles)


do_generate_csv()

## Celファイルを生成する

台形にするために範囲を変えて繰り返す


In [30]:
def convert_dot(micro_meter):
    dot = 50000
    area = 0.25  # 修論に書いてる数字の単位はmm^2
    length = math.sqrt(area) * 1000
    return int(micro_meter / (length / dot))


def compute_cel_row(rectangle):
    top_left_x = convert_dot(rectangle['top_left'][0])
    top_left_y = convert_dot(rectangle['top_left'][1])
    bottom_right_x = convert_dot(rectangle['bottom_right'][0])
    bottom_right_y = convert_dot(rectangle['bottom_right'][1])
    return f"\tTL{top_left_x}, {top_left_y}; TL{bottom_right_x}, {bottom_right_y}\n"


def generate_cel(rectangles):
    filename = input("Enter cel filename: ")
    rows = []
    for index, rectangle in enumerate(rectangles):
        if index == 0:
            rows.append("DS" + compute_cel_row(rectangle=rectangle))
        else:
            rows.append(compute_cel_row(rectangle=rectangle))
    with open(f"drawing_config/{filename}.cel", "w") as f:
        f.write("/*--- PARAMETER INITIALIZE ---*/;\nTS 1,1; TA 0;;\n")
        f.writelines(rows)
        f.write("END\n")


def do_generate_cel():
    rectangles = []
    polarization_period = 2.2
    rectangles.extend(do_compute_rectangles(polarization_period=polarization_period))
    polarization_period = 2.0
    rectangles.extend(do_compute_rectangles(polarization_period=polarization_period))
    polarization_period = 1.8
    rectangles.extend(do_compute_rectangles(polarization_period=polarization_period))
    generate_cel(rectangles)


do_generate_cel()